In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ! pip install datasets transformers
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cloud 0.1.13 requires tensorflow<3.0,>=1.15.0, which is not installed.
dask-cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda112, which is not installed.
cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda110, which is not installed.
s3fs 2021.6.1 requires fsspec==2021.06.1, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1.3.8 requires fsspec[http]!=2021.06.0,>=2021.05.0, but you have fsspec 2021.6.0 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires dask<=2021.5.1,>=2021.4.0, but you have dask 2021.6.2 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires distributed<=2021.5.1,>=2.22.0, but you have distributed 2021.6.2 which is incompatible.


In [3]:
from datasets import load_dataset, Dataset
from pprint import pprint
import pandas as pd
import numpy as np

# Preprocessing

In [4]:
# model_checkpoint = "distilbert-base-uncased"
# model_checkpoint = "deepset/xlm-roberta-large-squad2"
# model_checkpoint = "../input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2"
model_checkpoint = "../input/pipeline-for-qa-train-with-5-folds/chaii-trained-model-0"

from transformers import XLMTokenizer,AutoTokenizer
# tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
# tokenizer("What is your name?", "My name is Sylvain.")

# Doc_stride is used to handle large text: tokens>512

In [6]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

# Reading Data

In [7]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test_dataset = Dataset.from_pandas(test)

In [8]:
# test_dataset

# Preparing data

In [9]:
# In some padding required on left side
pad_on_right = tokenizer.padding_side == "right"
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [10]:
# features = prepare_validation_features(test_dataset[:5])
# print(features.keys())

In [11]:
 validation_features = test_dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=test_dataset.column_names
    )

In [12]:
# from tqdm.auto import tqdm
# import collections


# def postprocess_qa_predictions(examples, features, start_logits_cons, end_logits_cons, n_best_size = 20, 
#                                max_answer_length = 30):
#     all_start_logits = start_logits_cons
#     all_end_logits = end_logits_cons
# #     all_start_logits, all_end_logits = raw_predictions
#     # Build a map example to its corresponding features.
#     example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
#     features_per_example = collections.defaultdict(list)
#     for i, feature in enumerate(features):
#         features_per_example[example_id_to_index[feature["example_id"]]].append(i)

#     # The dictionaries we have to fill.
#     predictions = collections.OrderedDict()

#     # Logging.
#     print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

#     # Let's loop over all the examples!
#     for example_index, example in enumerate(tqdm(examples)):
#         # Those are the indices of the features associated to the current example.
#         feature_indices = features_per_example[example_index]

#         min_null_score = None # Only used if squad_v2 is True.
#         valid_answers = []
        
#         context = example["context"]
#         # Looping through all the features associated to the current example.
#         for feature_index in feature_indices:
#             # We grab the predictions of the model for this feature.
#             start_logits = all_start_logits[feature_index]
#             end_logits = all_end_logits[feature_index]
#             # This is what will allow us to map some the positions in our logits to span of texts in the original
#             # context.
#             offset_mapping = features[feature_index]["offset_mapping"]

#             # Update minimum null prediction.
#             cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
#             feature_null_score = start_logits[cls_index] + end_logits[cls_index]
#             if min_null_score is None or min_null_score < feature_null_score:
#                 min_null_score = feature_null_score

#             # Go through all possibilities for the `n_best_size` greater start and end logits.
#             start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
#             end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
#             for start_index in start_indexes:
#                 for end_index in end_indexes:
#                     # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
#                     # to part of the input_ids that are not in the context.
#                     if (
#                         start_index >= len(offset_mapping)
#                         or end_index >= len(offset_mapping)
#                         or offset_mapping[start_index] is None
#                         or offset_mapping[end_index] is None
#                     ):
#                         continue
#                     # Don't consider answers with a length that is either < 0 or > max_answer_length.
#                     if end_index < start_index or end_index - start_index + 1 > max_answer_length:
#                         continue

#                     start_char = offset_mapping[start_index][0]
#                     end_char = offset_mapping[end_index][1]
#                     valid_answers.append(
#                         {
#                             "score": start_logits[start_index] + end_logits[end_index],
#                             "text": context[start_char: end_char]
#                         }
#                     )
        
#         if len(valid_answers) > 0:
#             best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
#         else:
#             # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
#             # failure.
#             best_answer = {"text": "", "score": 0.0}
        
#         # Let's pick our final answer: the best one or the null answer (only for squad_v2)
# #         if not squad_v2:
#         predictions[example["id"]] = best_answer["text"]
# #         else:
# #             answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
# #             predictions[example["id"]] = answer

#     return predictions

# Fold predictions

In [13]:
import gc
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
gc.collect()
folds = 5

In [14]:
raw_predictions = []
for i in range(folds):
    print(f'predicting using model {i} start')
    model_checkpoint = f"../input/pipeline-for-qa-train-with-5-folds/chaii-trained-model-{i}"
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    trainer = Trainer(model)
    raw_predictions.append(trainer.predict(validation_features))
    print(f'prediction of model {i} complete')
    print('-----------------------------------')

predicting using model 0 start


prediction of model 0 complete
-----------------------------------
predicting using model 1 start


prediction of model 1 complete
-----------------------------------
predicting using model 2 start


prediction of model 2 complete
-----------------------------------
predicting using model 3 start


prediction of model 3 complete
-----------------------------------
predicting using model 4 start


prediction of model 4 complete
-----------------------------------


In [15]:
# len(raw_predictions)

In [16]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

# Ensemble results

In [17]:
start_logits_cons = 0
end_logits_cons = 0
for raw_prediction in raw_predictions:
    sc, ec = raw_prediction.predictions
    sc = sc/folds
    ec = ec/folds
    start_logits_cons += sc
    end_logits_cons += ec
print(len(start_logits_cons))

67


In [18]:
# start_logits_cons

# Model 2

In [19]:
import os
import gc
gc.enable()
import math
import json
import time
import random
import multiprocessing
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
from torch.utils.data.distributed import DistributedSampler

try:
    from apex import amp
    APEX_INSTALLED = True
except ImportError:
    APEX_INSTALLED = False

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    logging,
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
)
logging.set_verbosity_warning()
logging.set_verbosity_error()

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def optimal_num_of_loader_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

print(f"Apex AMP Installed :: {APEX_INSTALLED}")
MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

Apex AMP Installed :: False


In [20]:
class Config:
    # model
    model_type = 'xlm_roberta'
    model_name_or_path = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2/"
    config_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2/"
    fp16 = True if APEX_INSTALLED else False
    fp16_opt_level = "O1"
    gradient_accumulation_steps = 2

    # tokenizer
    tokenizer_name = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2/"
    max_seq_length = 384
    doc_stride = 128

    # train
    epochs = 1
    train_batch_size = 4
    eval_batch_size = 32#8

    # optimizer
    optimizer_type = 'AdamW'
    learning_rate = 1.5e-5
    weight_decay = 1e-2
    epsilon = 1e-8
    max_grad_norm = 1.0

    # scheduler
    decay_name = 'linear-warmup'
    warmup_ratio = 0.1

    # logging
    logging_steps = 10

    # evaluate
    output_dir = 'output'
    seed = 2021
    
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [21]:
class Model(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(Model, self).__init__()
        self.config = config
        self.xlm_roberta = AutoModel.from_pretrained(modelname_or_path, config=config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self._init_weights(self.qa_outputs)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
    ):
        outputs = self.xlm_roberta(
            input_ids,
            attention_mask=attention_mask,
        )

        sequence_output = outputs[0]
        pooled_output = outputs[1]
        
        # sequence_output = self.dropout(sequence_output)
        qa_logits = self.qa_outputs(sequence_output)
        
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
        return start_logits, end_logits

In [22]:
def make_model(args):
    config = AutoConfig.from_pretrained(args.config_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    model = Model(args.model_name_or_path, config=config)
    return config, tokenizer, model

In [23]:
def prepare_test_features(args, example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=args.max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['id']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

In [24]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
base_model_path = '../input/chaii-xlmroberta-large-v6/output/'

tokenizer = AutoTokenizer.from_pretrained(Config().tokenizer_name)

test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(Config(), row, tokenizer)

args = Config()
test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=args.eval_batch_size, 
    sampler=SequentialSampler(test_dataset),
    num_workers=optimal_num_of_loader_workers(),
    pin_memory=True, 
    drop_last=False
)

In [25]:
def get_predictions(checkpoint_path):
    config, tokenizer, model = make_model(Config())
    print(f'model loading: {checkpoint_path}')
    model.cuda();
    model.load_state_dict(
        torch.load(base_model_path + checkpoint_path)
    );
    
    start_logits = []
    end_logits = []
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    del model, tokenizer, config
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [26]:
start_logits1, end_logits1 = get_predictions('checkpoint-fold-0/pytorch_model.bin')
start_logits2, end_logits2 = get_predictions('checkpoint-fold-1/pytorch_model.bin')
start_logits3, end_logits3 = get_predictions('checkpoint-fold-2/pytorch_model.bin')
start_logits4, end_logits4 = get_predictions('checkpoint-fold-3/pytorch_model.bin')
start_logits5, end_logits5 = get_predictions('checkpoint-fold-4/pytorch_model.bin')

start_logits = (start_logits1 + start_logits2 + start_logits3 + start_logits4 + start_logits5) / 5
end_logits = (end_logits1 + end_logits2 + end_logits3 + end_logits4 + end_logits5) / 5

model loading: checkpoint-fold-0/pytorch_model.bin


100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


model loading: checkpoint-fold-1/pytorch_model.bin


100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


model loading: checkpoint-fold-2/pytorch_model.bin


100%|██████████| 3/3 [00:04<00:00,  1.56s/it]


model loading: checkpoint-fold-3/pytorch_model.bin


100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


model loading: checkpoint-fold-4/pytorch_model.bin


100%|██████████| 3/3 [00:03<00:00,  1.32s/it]


# Ensembled logits

In [27]:
# normalize
normalizer = torch.nn.Softmax(dim=1)

start_logits = normalizer(torch.from_numpy(start_logits))
end_logits = normalizer(torch.from_numpy(end_logits))

start_logits = start_logits.numpy()
end_logits = end_logits.numpy()


start_logits_cons = normalizer(torch.from_numpy(start_logits_cons))
end_logits_cons = normalizer(torch.from_numpy(end_logits_cons))

start_logits_cons = start_logits_cons.numpy()
end_logits_cons = end_logits_cons.numpy()

In [28]:
model1 = 0.4#0.5
model2 = 0.6#0.5

final_start_logits = model1*start_logits_cons + model2*start_logits
final_end_logits = model1*end_logits_cons + model2*end_logits

# Preparing final results

In [29]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
        
    return predictions

In [30]:
# final_predictions = postprocess_qa_predictions(test_dataset, validation_features, final_start_logits, final_end_logits)
# final_predictions = postprocess_qa_predictions(test_dataset, validation_features, start_logits_cons, end_logits_cons)
predictions = postprocess_qa_predictions(test, test_features, (final_start_logits, final_end_logits))


Post-processing 5 example predictions split into 67 features.


In [31]:
test['PredictionString'] = test['id'].map(predictions)

In [32]:
test.head()

,id,context,question,language,PredictionString
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi,येलन
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi,20 अप्रैल 2010
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi,१२ मार्च १८२४
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil,13
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


# Custom Post-Processing

In [33]:
bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"


cleaned_preds = []
for pred, context in test[["PredictionString", "context"]].to_numpy():
    if pred == "":
        cleaned_preds.append(pred)
        continue
    while any([pred.startswith(y) for y in bad_starts]):
        pred = pred[1:]
    while any([pred.endswith(y) for y in bad_endings]):
        if pred.endswith("..."):
            pred = pred[:-3]
        else:
            pred = pred[:-1]
    
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

test["PredictionString"] = cleaned_preds

In [34]:
test.head()

,id,context,question,language,PredictionString
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi,येलन
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi,20 अप्रैल 2010
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi,१२ मार्च १८२४
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil,13
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


In [35]:
test[['id','PredictionString']].to_csv('submission.csv',index=False)